## Notebook to test the impact of a midatlantic ridge

In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import nc_time_axis

#plt.rcParams['axes.facecolor']='black'
#plt.style.use('dark_background')
plt.style.use('default')

plt.rcParams.update({'font.size': 20})

In [ ]:
path   = "/data/dkamm/nemo_output/NEVERWORLD/"

## Load domain (not lazy)

In [ ]:
from pathlib import Path
import xnemogcm as xn

In [ ]:
datadir_gaus    = Path(path + 'GAUS_RIDGE_init/restart1')
datadir_no      = Path(path + 'NO_RIDGE_init/restart1')

In [ ]:
domcfg_gaus = xn.open_domain_cfg(datadir=datadir_gaus)
domcfg_no   = xn.open_domain_cfg(datadir=datadir_no  )

### 1. Bathymetry without mid-atlantic ridge

In [ ]:
%matplotlib widget
fig = plt.figure()
ax  = plt.axes(projection='3d')
ax.set_box_aspect(aspect = (1,2.5,1))
p1 = (-domcfg_no.e3w_0.sum('z_f')).plot.surface(ax=ax)
ax.set_title('')

### 2. Bathymetry with mid-atlantic ridge:      Gaussian edge 

In [ ]:
%matplotlib widget
fig = plt.figure()
ax  = plt.axes(projection='3d')
ax.set_box_aspect(aspect = (1,2.5,1))
p2 = (-domcfg_gaus.e3w_0.sum('z_f')).plot.surface(ax=ax)
ax.set_title('')

In [ ]:
import numpy as np

def gauss(rn_slp, rn_edge_n, phi, h_top, h_bot):
    if(h_bot >= h_top):
        gauss = (h_top - h_bot) * np.exp( - ( ( phi - rn_edge_n ) / rn_slp ) ** 2 ) + h_bot
    else:
        gauss = h_bot
    return gauss

In [ ]:
Data_no = xn.open_nemo(domcfg=domcfg_no, files=[
        datadir_no / '../restart7/NEVERWORLD_grid_T.nc',
        datadir_no / '../restart7/NEVERWORLD_grid_U.nc',
        datadir_no / '../restart7/NEVERWORLD_grid_V.nc'
    ])

Data_gaus = xn.open_nemo(domcfg=domcfg_gaus, files=[
        datadir_gaus / '../restart7/NEVERWORLD_grid_T.nc',
        datadir_gaus / '../restart7/NEVERWORLD_grid_U.nc',
        datadir_gaus / '../restart7/NEVERWORLD_grid_V.nc'
    ])

In [ ]:
Data_no_mo = xn.open_nemo(domcfg=domcfg_no, files=[
        datadir_no / '../restart7/NEVERWORLD_grid_T_mo.nc'
    ])

Data_gaus_mo = xn.open_nemo(domcfg=domcfg_gaus, files=[
        datadir_gaus / '../restart7/NEVERWORLD_grid_T_mo.nc'
    ])

### Monthly averaged MLD

In [ ]:
Data_no_mo.mldr10_1.isel(t=slice(-120,-1)).groupby("t.month").mean().plot(col='month', col_wrap=4)
plt.suptitle('Monthly average over the last ten years: No Mid-Atlantic ridge', y=1.04)

In [ ]:
Data_gaus_mo.mldr10_1.isel(t=slice(-120,-1)).groupby("t.month").mean().plot(col='month', col_wrap=4)
plt.suptitle('Monthly average over the last ten years: With Mid-Atlantic ridge', y=1.04)

### Barotropic Streamfunction

In [ ]:
metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t_0', 'e3u_0', 'e3v_0', 'e3f_0'] # Z distances
}

grid_no = xgcm.Grid(domcfg_no,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

grid_gaus = xgcm.Grid(domcfg_gaus,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

In [ ]:
#Interpolating on u_f
Data_no['u_f'] = grid_no.interp(Data_no.uoce, 'Y')
Data_gaus['u_f'] = grid_gaus.interp(Data_gaus.uoce, 'Y')

#Integrating over depth
Data_no['U'] = grid_no.integrate(Data_no.u_f, 'Z')
Data_gaus['U'] = grid_gaus.integrate(Data_gaus.u_f, 'Z')

#Cumulative integral over x
Psi_no = (Data_no.U[-1,::-1,:] * domcfg_no.e2f[::-1,:]).cumsum('y_f') / 1e6
Psi_gaus = (Data_gaus.U[-1,::-1,:] * domcfg_gaus.e2f[::-1,:]).cumsum('y_f') / 1e6

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,8))
a = Psi_no.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False, ax=axs[1])
cbar1 = fig.colorbar(a, ax=axs[:2], location='bottom', label=r'$\psi$ [Sv]')
b = Psi_gaus.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False, ax=axs[0], vmin=cbar1.vmin, vmax=cbar1.vmax)
axs[0].set_ylabel('latitude [°N]')
axs[1].set_ylabel('latitude [°N]')
axs[0].set_xlabel('longitude [°E]')
axs[1].set_xlabel('longitude [°E]')
axs[0].set_title('With mid-atlantic ridge')
axs[1].set_title('Without mid-atlantic ridge')

In [ ]:
Psi_gaus = Psi_gaus.assign_coords(dict(
    x_sh=(["y_f", "x_f"], Psi_gaus.glamf.values - 60.)
    ))
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = Psi_gaus.plot.contourf(x='x_sh', y='gphif', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
axs.set_ylabel('latitude [°N]')
axs.set_xlabel('longitude [°E]')
plt.title('')
axs.set_facecolor('none')
fig.savefig('BSF.png', transparent=True)

In [ ]:
Psi_gaus

In [ ]:
# U_s_grid = U_s_grid.assign_coords({'x_f': U_s_grid.x_f.values - 60.})
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = Psi_1_4deg.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
axs.set_ylabel('latitude [°N]')
axs.set_xlabel('longitude [°E]')
plt.title('')

### Cross-equatorial meridional velocities

In [ ]:
Data_no = Data_no.assign_coords(dict(
    depthv=(["z_c", "y_f", "x_c"], - Data_no.isel(t=-1).e3v.cumsum('z_c').values)
    )
)

Data_gaus = Data_gaus.assign_coords(dict(
    depthv=(["z_c", "y_f", "x_c"], - Data_gaus.isel(t=-1).e3v.cumsum('z_c').values)
    )
)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,8), sharey=True)
a = (Data_no.vocetr_eff.isel(t=-1, y_f=100) / 10e6).plot.contourf(x='x_c', y='depthv', cmap='RdBu_r', levels=30, add_colorbar=False, ax=axs[1])
cbar1 = fig.colorbar(a, ax=axs[:2], location='bottom', label=r'$\psi$ [Sv]')
b = (Data_gaus.vocetr_eff.isel(t=-1, y_f=100) / 10e6).plot.contourf(x='x_c', y='depthv', cmap='RdBu_r', levels=30, add_colorbar=False, ax=axs[0], vmin=cbar1.vmin, vmax=cbar1.vmax)
axs[0].set_ylabel('latitude [°N]')
axs[1].set_ylabel('latitude [°N]')
axs[0].set_facecolor('grey')
axs[1].set_facecolor('grey')
axs[1].set_ylabel('latitude [°N]')
axs[0].set_xlabel('longitude [°E]')
axs[1].set_xlabel('longitude [°E]')
axs[0].set_title('With mid-atlantic ridge')
axs[1].set_title('Without mid-atlantic ridge')

### ACC strength

In [ ]:
n_restarts = 8

In [ ]:
#Combine restart files
for i in range(n_restarts-2, 0, -1):
    _datadir = Path(path + 'NO_RIDGE_init/restart{}/'.format(i))
    _Data = xn.open_nemo(domcfg=domcfg_no, files=[
        _datadir / 'NEVERWORLD_grid_T.nc',
        _datadir / 'NEVERWORLD_grid_U.nc',
        _datadir / 'NEVERWORLD_grid_V.nc'
    ])
    Data_no = xr.concat([_Data.chunk(), Data_no.chunk()], "t")

for i in range(n_restarts-2, 0, -1):
    _datadir = Path(path + 'GAUS_RIDGE_init/restart{}/'.format(i))
    _Data = xn.open_nemo(domcfg=domcfg_gaus, files=[
        _datadir / 'NEVERWORLD_grid_T.nc',
        _datadir / 'NEVERWORLD_grid_U.nc',
        _datadir / 'NEVERWORLD_grid_V.nc'
    ])
    Data_gaus = xr.concat([_Data.chunk(), Data_gaus.chunk()], "t")

In [ ]:
#Compute ACC transport
ACC_no = Data_no.isel(x_f=0).uocetr_eff.sum(['y_c', 'z_c']) / 1e6
ACC_gaus = Data_gaus.isel(x_f=0).uocetr_eff.sum(['y_c', 'z_c']) / 1e6

#OR!!!!
#ACC_no = (Data_no.isel(x_f=0).uoce * Data_no.isel(x_f=0).e2u * Data_no.isel(x_f=0).e3u).sum(['y_c', 'z_c']) / 1e6

In [ ]:
import cftime as cft
ACC_no = ACC_no.assign_coords(dict({'t':cft.to_tuple(ACC_no.indexes['t'])[0]}))
ACC_gaus = ACC_gaus.assign_coords(dict({'t':cft.to_tuple(ACC_gaus.indexes['t'])[0]}))

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(12,6))
ACC_no.plot(color='midnightblue', label='Without mid-atlantic ridge')
ACC_gaus.plot(color='darkred', label='With mid-atlantic ridge')
axs.set_ylabel('ACC transport [Sv]')
plt.title('')
plt.xlabel('time [years]')
plt.legend()
plt.grid()

In [ ]:
!jupyter nbconvert --to html ./Mid_Atlantic_Ridge.ipynb --HTMLExporter.theme=dark 

### Monthly averaged MLD

In [ ]:
Data_gaus_mo.qsr.isel(y_c=150, x_c=30, t=slice(0,24)).plot()

In [ ]:
Data_gaus_mo.qsr.isel(t=slice(-13,-1), x_c=30, y_c=175).plot()
#plt.suptitle('Monthly average over the last ten years: With Mid-Atlantic ridge', y=1.04)

In [ ]:
Data_gaus

In [ ]:
Data_gaus_mo

In [ ]:
a = xr.open_dataset(path + 'NEVERWORLD_grid_T_mo.nc')

In [ ]:
a.nav_lat.isel(y=175).values

In [ ]:
a.qsr.plot(col='time_counter', col_wrap=3)

### Barotropic Streamfunction